In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#data collection
dataset=pd.read_csv("Social_Network_Ads.csv")
print(dataset)

#preprocess
#drop/remove user ID, model creation
dataset=dataset.drop("User ID",axis=1)
dataset

#preprocess Gender to 0 or 1
dataset=pd.get_dummies(dataset,drop_first=True).astype(int)
print(dataset.columns)

#Balanced Dataset or Imbalanced Dataset
print(dataset["Purchased"].value_counts())

#input output split
independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent=dataset[['Purchased']]

print(independent.shape)
print(dependent.shape)

#Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent, dependent, test_size=0.30,random_state=0)

#preprocess standard Scaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

      User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
395  15691863  Female   46            41000          1
396  15706071    Male   51            23000          1
397  15654296  Female   50            20000          1
398  15755018    Male   36            33000          0
399  15594041  Female   49            36000          1

[400 rows x 5 columns]
Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')
Purchased
0    257
1    143
Name: count, dtype: int64
(400, 3)
(400, 1)


In [2]:
#Model creation
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
from sklearn.ensemble import RandomForestClassifier

#Model creation using Grid
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]}
classifier = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
# fitting the model for grid search 
classifier.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 436, in _validate_params
    validate_parameter_constraints(
    ~~~~~~~~~~~~~

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [3]:
#Evaluation Matrix: Confusion_matrix
re=classifier.cv_results_
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
#Evaluation Matrix: calculate P R F
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)
print(clf_report)

[[74  5]
 [ 5 36]]
              precision    recall  f1-score   support

           0       0.94      0.94      0.94        79
           1       0.88      0.88      0.88        41

    accuracy                           0.92       120
   macro avg       0.91      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [4]:
# print best parameter after tuning 
#print(classifier.best_params_) 
re=classifier.cv_results_
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter {}:".format(classifier.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 100}: 0.9166666666666666


In [5]:
#Roc = ROC (Receiver Operating Characteristic) Curve
#AUC (Area Under the Curve)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,classifier.predict_proba(X_test)[:,1])

np.float64(0.9694350108058042)

In [6]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002006,0.000548,0.000000,0.000000,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.001856,0.000515,0.000000,0.000000,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.043496,0.007289,0.014354,0.002493,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.855314,0.840114,0.840114,0.947015,0.868182,0.870148,0.039841,7
3,0.280740,0.009165,0.024370,0.001049,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.874254,0.858503,0.823129,0.911105,0.946153,0.882629,0.042517,3
4,0.033515,0.002030,0.013052,0.001066,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.874254,0.840114,0.804584,0.911105,0.926743,0.871360,0.044924,6
5,0.258753,0.006300,0.024074,0.002384,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.874254,0.819142,0.859435,0.911105,0.963889,0.885565,0.049009,2
6,0.000865,0.000031,0.000000,0.000000,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.001154,0.000408,0.000000,0.000000,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.031352,0.001903,0.014145,0.001853,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.874254,0.838326,0.823129,0.891667,0.946153,0.874706,0.043304,5
9,0.249477,0.006893,0.022888,0.000570,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.874254,0.857143,0.841398,0.911105,0.946153,0.886011,0.037970,1


In [7]:
#save model
import pickle
filename="final_Sav_Model.sav"
pickle.dump(classifier,open(filename,'wb'))
load_model=pickle.load(open("final_Sav_Model.sav",'rb'))

#apply StandardScaler to user input
preinput=sc.transform([[46,41000,1]])
print(preinput)
result=load_model.predict(preinput)
result

[[ 0.79762394 -0.8330751   0.99288247]]


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([1])